# Getting Started with Shape Cohort Generator

## Before you start!

- This notebook assumes that shapeworks conda environment has been activated using `conda activate shapeworks` on the terminal.
- See [Setting Up ShapeWorks Environment](setting-up-shapeworks-environment.ipynb) to learn how to set up your environment to start using shapeworks library. Please note, the prerequisite steps will use the same code to setup the environment for this notebook and import `shapeworks` library.


## In this notebook, you will learn:

1. How to use the `ShapeCohortGenerator` package to generate meshes and segmentation images for <br>
   i. Ellipsoids <br>
   ii. Supershapes


We will also define modular/generic helper functions as we walk through these items to reuse functionalities without duplicating code.


## Prerequisites

- Setting up `shapeworks` environment. See [Setting Up ShapeWorks Environment](setting-up-shapeworks-environment.ipynb). To avoid code clutter, the `setup_shapeworks_env` function can found in `Examples/Python/setupenv.py` module. 
- Importing `shapeworks` library. See [Setting Up ShapeWorks Environment](setting-up-shapeworks-environment.ipynb).

## Note about shapeworks APIs

shapeworks functions are inplace, i.e., `<swObject>.<function>()` applies that function to the `swObject` data. To keep the original data unchanged, you have first to copy it to another variable before applying the function.

## Notebook keyboard shortcuts

- `Esc + H`: displays a complete list of keyboard shortcuts
- `Esc + A`: insert new cell above the current cell
- `Esc + B`: insert new cell below the current cell
- `Esc + D + D`: delete current cell
- `Esc + Z`: undo
- `Shift + enter`: run current cell and move to next
- To show a function's argument list (i.e., signature), use `(` then `shift-tab`
- Use `shift-tab-tab` to show more help for a function
- To show the help of a function, use `help(function)` or `function?`
- To show all functions supported by an object, use `dot-tab` after the variable name

## Prerequisites

### Setting up shapeworks environment 

Here, we will append both your `PYTHONPATH` and your system `PATH` to setup shapeworks environment for this notebook. See [Setting Up ShapeWorks Environment](setting-up-shapeworks-environment.ipynb) for more details.

In this notebook, we assume the following.

- This notebook is located in `Examples/Python/notebooks/tutorials`
- You have built shapeworks in `build` directory within the shapeworks code directory
- You have built shapeworks dependencies (using `build_dependencies.sh`) in the same parent directory of shapeworks code

In [ ]:
# import relevant libraries 
import sys 
import os
import itkwidgets as itkw
import pyvista as pv
# add parent-parent directory (where setupenv.py is) to python path
sys.path.insert(0,'../..')

# importing setupenv from Examples/Python

import setupenv

# indicate the bin directories for shapeworks and its dependencies
shapeworks_bin_dir   = "/home/sci/iyerkrithika/ShapeWorks/build_gs/bin"
dependencies_bin_dir = "/home/sci/iyerkrithika/ShapeWorks/dependencies/install/bin"

# set up shapeworks environment
setupenv.setup_shapeworks_env(shapeworks_bin_dir,  
                              dependencies_bin_dir, 
                              verbose = True)

### Importing `shapeworks` library

In [ ]:
# let's import shapeworks library to test whether shapeworks is now set
try:
    import shapeworks as sw
except ImportError:
    print('ERROR: shapeworks library failed to import')
else:
    print('SUCCESS: shapeworks library is successfully imported!!!')

### Importing `ShapeCohortGen` library

In [ ]:
# let's import ShapeCohortGen library 
try:
    import ShapeCohortGen
except ImportError:
    print('ERROR: ShapeCohortGen library failed to import')
else:
    print('SUCCESS: ShapeCohortGen library is successfully imported!!!')


### Defining a helper function

As converting between shapeworks Mesh object and vtk mesh is a step that we will need frequently, let's add a helper function for this purpose.

In [ ]:
# a helper function that converts shapeworks Mesh object to vtk mesh 
# TODO: to be modifed when #825 is addressed


def sw2vtkMesh(swMesh, verbose = True):
    
    if verbose:
        print('Header information: ')
        print(swMesh)

    # save mesh
    swMesh.write('temp.vtk')

    # read mesh into an itk mesh data
    vtkMesh = pv.read('temp.vtk')
    
    # remove the temp mesh file
    os.remove('temp.vtk')
    
    return vtkMesh

### Need to add more info here
This shape cohort generator package is capable of generating ellipsoids and supershapes. This package generates the meshes first and then generates segmentaion images for the corresponding meshes. 



## 1. Generating Ellipsoid Dataset

### 1. Mesh Generation

Initialize an ellipsoid cohort generator

In [ ]:
ellipsoid_generator = ShapeCohortGen.EllipsoidCohortGenerator()

Each generator has two functions:
1. `generate()` - mesh generation
2. `generate_segmentations()` - segmentation generation
The mesh generation fuction has to be called first before generating segmentations

### For the ellipsoid mesh generation, you can specify the following arguments:
1. `num_samples` - number of samples in the cohort(dataset)<br>
    Datatype : `int` <br>
    <br>
2. `out_dir` - path where the dataset should be saved<br>
    Datatype : `string`<br> 
    <br>
    
3. `randomize_center` - randomizes the centers for ellipsoid mesh generation if set to `True`<br>
    Datatype : `bool` <br> 
    Defaut value : `True`
    <br>
4. `randomize_rotation` - randomizes the orientation of the ellispoids if set to `True` <br>
    Datatype : `bool` <br> 
    Defaut value : `True`
    <br>

# out_dir does not work ?

In [ ]:
num_samples = 15
out_dir = '../out/'
meshFiles = ellipsoid_generator.generate(num_samples,out_dir)

# Does not work with .ply files
Read a few meshes and display them

In [ ]:
vtkMeshList = []
for i in range(len(meshFiles)):
    shapeMesh_vtk = sw2vtkMesh(sw.Mesh(meshFiles[i]),verbose=False)
    vtkMeshList.append(shapeMesh_vtk)
    
    
    
# visualize with axes and auto rotation
itkw.view(  geometries       = vtkMeshList, 
            rotate           = True, # enable auto rotation
            axes             = True)

### 2. Segmentation Generation

### For the ellipsoid mesh generation, you can specify the following arguments:

1. `meshList` - List of meshes to be rasterized for segmentation image generation. This is returned by the `generate` function (see above)
<br>

2. `out_dir` - path where the dataset should be saved<br>
    Datatype : `string`<br> 
    <br>
3. `randomize_size` - randomize the size of the images to include more background if set to `True`<br>
    Datatype : `bool` <br> 
    Defaut value : `True`
    <br>
4. `spacing` - ADD INFO
    <br>
5. `allow_on_boundary` - If set to `True`,randomly selects 20% samples and ensure that the shapes are touching two random selected axes out of `[x,y,z]`<br>
    Datatype : `bool` <br> 
    Defaut value : `True`
    <br>

In [ ]:
segFiles = ellipsoid_generator.generate_segmentations(out_dir)

## Discuss how to visualize segs and meshes  #TODO